In [ ]:
import geopandas as gpd
import networkx as nx
import momepy
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json
from shapely.geometry import Point, LineString
from shapely.ops import split, unary_union, nearest_points
from shapely.geometry import GeometryCollection, MultiPoint
from road_adder import *

from transport_frames.utils.helper_funcs import convert_list_attr_from_str, prepare_graph
from transport_frames.indicators.utils import availability_matrix

from transport_frames.road_adder.road_adder import add_roads





/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
citygraph = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/Ленинградская_область/Ленинградская_область_uds.graphml')
citygraph = convert_list_attr_from_str(citygraph)
citygraph = prepare_graph(citygraph)
n,e = momepy.nx_to_gdf(citygraph)



Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10809e530>>
Traceback (most recent call last):
  File "/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
nu = gpd.read_file('/Users/polina/Desktop/citygraph_e_new_edges.geojson').to_crs(32636)

## run

In [ ]:
graf = add_roads(citygraph,nu,n,e,32636,node_buffer = 300,road_buffer = 5000)

104070
104071


/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:348: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_nodes = pd.concat([added_nodes, new_end_node]) # adding ends
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:354: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_edges = pd.concat([added_edges, bridge_edge]) # if we can connect end to a node, only the bridge is added


104072


/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:429: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_nodes = pd.concat([added_intersections_nodes,new_intersection]) # adding new no
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:441: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_edges = pd.concat([added_intersections_edges,new_piece]) # adding road pieces


104073
104074
104075


...

/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:429: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_nodes = pd.concat([added_intersections_nodes,new_intersection]) # adding new no
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:441: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_edges = pd.concat([added_intersections_edges,new_piece]) # adding road pieces


104102
104103


/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:348: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_nodes = pd.concat([added_nodes, new_end_node]) # adding ends
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:354: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_edges = pd.concat([added_edges, bridge_edge]) # if we can connect end to a node, only the bridge is added


104104
104105
104106
104107
104108
104109
104110


/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:434: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_nodes = pd.concat([added_intersections_nodes,new_intersection]) # adding new nodes
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/road_adder/road_adder.py:441: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  added_intersections_edges = pd.concat([added_intersections_edges,new_piece]) # adding road piec

## plots

In [ ]:
# для каждой новой дороги:
#     для каждого конца дороги:
#     new_nodes.add(конец)
#         если рядом вершина:
#             new_edges.add(ребро от конца до вершины)
#         если рядом дорога:
#             new_nodes.add(ближайшая точка на дороге)
#             del_edges.add(дорога)
#             new_edges.add(ребро от конца до вершины)

In [ ]:
n_,e_ = momepy.nx_to_gdf(graf)
n_ = n_[n_.intersects(point2.buffer(10000))]
e_ = e_[e_.intersects(point2.buffer(10000))]
e_.flag = e_['flag'].fillna('original')

In [ ]:
w.explore('flag')

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
from folium import features



# Combine edges and nodes into a single GeoDataFrame
w = pd.concat([e, n]).to_crs(4236)

# Apply the custom color function

# Initialize the Folium map
m = folium.Map(location=[w.geometry.centroid.y.mean(), w.geometry.centroid.x.mean()], zoom_start=12)

# Add lines to the map
for _, row in w[w.geometry.type == 'LineString'].iterrows():
    folium.PolyLine(
        locations=[(p[1], p[0]) for p in row['geometry'].coords],
        color='grey',
        weight=3
    ).add_to(m)

# Add markers to the map
for _, row in w[w.geometry.type == 'Point'].iterrows():
    folium.CircleMarker(
        location=(row['geometry'].y, row['geometry'].x),
        color='grey',
        fill=True,
        fill_opacity=0.5,
        radius=5
    ).add_to(m)

# Save the map to an HTML file

# Display the map
m


In [ ]:
line_gdf['maxspeed'] = 110/3.6
line_gdf

In [ ]:
# связность + выделенная новая дорога на карте ЛО
# улучшение показателей
# Вход выход
# схемка?
# вид вблизи (подкрашены ноды)

In [ ]:
c1.to_service.median(), c2.to_service.median()

In [ ]:
poin = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/points.geojson').to_crs(local_crs)
poin['points'] = 1
c1 = indicators.new_connectivity(citygraph, poin, local_crs=local_crs, inter=False)
c2 = indicators.new_connectivity(graf, poin, local_crs=local_crs, inter=False)

In [ ]:
district = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/polygons18.geojson').to_crs(local_crs)
settlement = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/polygons188.geojson').to_crs(local_crs)

res = gpd.sjoin(c1, settlement, how="left", predicate="within").groupby('index_right').median(['to_service'])
c11 = pd.merge(settlement.reset_index(), res, left_on='index', right_on='index_right')
res = gpd.sjoin(c2, settlement, how="left", predicate="within").groupby('index_right').median(['to_service'])
c22 = pd.merge(settlement.reset_index(), res, left_on='index', right_on='index_right')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
c11.plot(column='to_service', cmap=plt.get_cmap('RdYlGn_r'), ax=ax,  legend=True, markersize=4).set_axis_off()
nodes, edges = n,e
e_[e_.reg==1].plot(ax=ax, color='white', label='Федеральные трассы')
e_[e_.flag=='line_segment'].plot(ax=ax, color='black', label='Новая дорога')
colorbar = plt.colorbar(cm.ScalarMappable(cmap=plt.get_cmap('RdYlGn_r'), norm=plt.Normalize(vmin=c11['to_service'].min(), vmax=c11['to_service'].max())), ax=ax)
# colorbar.set_ticks(np.arange(0, c11['to_service'].max(), 0.5))
ax.legend(loc='lower right')
fig.delaxes(fig.axes[1])
# plt.title('Оценка территорий на каркасе')
plt.savefig('12.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
c22.plot(column='to_service', cmap=plt.get_cmap('RdYlGn_r'), ax=ax,  legend=True, markersize=4).set_axis_off()
nodes, edges = n,e
e_[e_.reg==1].plot(ax=ax, color='white', label='Федеральные трассы')
e_[e_.flag=='line_segment'].plot(ax=ax, color='black', label='Новая дорога')
colorbar = plt.colorbar(cm.ScalarMappable(cmap=plt.get_cmap('RdYlGn_r'), norm=plt.Normalize(vmin=c22['to_service'].min(), vmax=c22['to_service'].max())), ax=ax)
# colorbar.set_ticks(np.arange(0, c11['to_service'].max(), 0.5))
ax.legend(loc='lower right')
fig.delaxes(fig.axes[1])
# plt.title('Оценка cdzp')
plt.savefig('11.png')
plt.show()

In [ ]:
e